# TODO

1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
 - 문서가 길면 토큰수 초과로 답변을 생성하지 못할 수 있다.
 - 문서가 길면 (인풋이 길면) 답변 생성에 시간이 너무 많이 소요된다.
 3. 임베딩 -> 벡터 데이터베이스에 저장
 4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [12]:
#%pip install --upgrade --quiet  docx2txt langchain-community

In [ ]:
%pip install python -m python-dotenv langchain langchain-upstage langchain-comunity langchain-text-splitters

## Load and Split

In [34]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import CharacterTextSplitter

# text splitter는 보통 두개의 파라미터를 가짐. 1. 청크사이즈, 2. 청크오버랩
text_splitter = CharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("tax.docx")
# document = loader.load()
# document를 loader.load()대신 loader.load_and_split(text_splitter)으로 불러오면 청크사이즈와 청크오버랩을 고려해서 쪼개진 문서를 반환함.
document_list = loader.load_and_split(text_splitter=text_splitter)

In [35]:
len(document_list)

182

In [ ]:
# %pip install -qU langchain_text_splitters

Note: you may need to restart the kernel to use updated packages.


## Embedding

In [3]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")



ValueError: You must specify an api key. You can pass it an argument as `api_key=...` or set the environment variable `UPSTAGE_API_KEY`.

이제 우리가 임베딩 한 결과를 VectorDB에 넣어주어야함.

In [21]:
#%pip install langchain-chroma

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 8.8 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 7.8 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.9 MB/s  0:00:00 eta 0

쪼개놓은 결과인 document_list를 embedding모델을 통해 VectorDB에 저장

In [ ]:
from langchain_chroma import Chroma

# ChromaDB는 기본적으로 인메모리 데이터베이스여서 창을 닫으면 다 날아감. 그래서 persist_directory를 설정해주면 창을 닫더라도 날아기 않음
database = Chroma.from_documents(documents=document_list, embedding=embeddings, collection_name='chroma-tax', persist_directory = './chroma', )


In [38]:
query = "연봉 5000만원의 직장인은 소득세가 얼마인가요?"
# 이건 실무 잡이야기이긴 한데, 존댓말로 물어보면 답변을 더 잘해준다고 합니다. 영어질문은 Please 붙여서 하면 더 답변이 잘나온다는 이야기가 있습니다.
retrieved_docs = database.similarity_search(query, k=3) # similarity_search의 디폴트 값이 4이므로 k값 설정을 안해두면 4개의 문서를 반환할 것임.

In [39]:
retrieved_docs

[Document(id='b641e9af-a61f-4d75-963a-bdde8141c6a5', metadata={'source': 'tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다)

In [ ]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [42]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}

"""

In [32]:
ai_message = llm.invoke(prompt)

In [33]:
ai_message.content

'연봉 5,000만원의 직장인이 납부해야 할 소득세는 여러 요소에 따라 달라질 수 있습니다. 기본적으로 소득세는 과세표준에 따라 세율이 적용되며, 공제 항목도 고려해야 합니다. \n\n2023년 기준으로, 연봉 5,000만원에 대한 소득세 계산은 다음과 같은 단계로 진행됩니다:\n\n1. **과세표준 계산**: 연봉에서 기본 공제(예: 인적 공제, 근로소득 공제 등)를 차감하여 과세표준을 계산합니다.\n   \n2. **세율 적용**: 과세표준에 따라 세율을 적용합니다. 2023년 기준으로 소득세율은 다음과 같습니다:\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 4,600만원 이하: 15%\n   - 4,600만원 초과 8,800만원 이하: 24%\n\n3. **세액 계산**: 각 구간에 해당하는 소득에 대해 세율을 적용하여 세액을 계산합니다.\n\n예를 들어, 연봉 5,000만원의 경우:\n- 1,200만원까지: 1,200만원 × 6% = 72만원\n- 1,200만원 초과 4,600만원까지: (4,600만원 - 1,200만원) × 15% = 51만원\n- 4,600만원 초과 5,000만원까지: (5,000만원 - 4,600만원) × 24% = 96만원\n\n이렇게 계산하면:\n- 72만원 + 51만원 + 96만원 = 219만원\n\n4. **최종 세액**: 위의 계산에서 기본 공제나 추가 공제를 적용하여 최종 세액을 결정합니다.\n\n따라서, 연봉 5,000만원의 직장인은 대략적으로 219만원의 소득세를 납부해야 할 것으로 예상됩니다. 하지만, 개인의 상황에 따라 추가 공제나 세액 공제가 있을 수 있으므로, 정확한 세액은 세무 전문가와 상담하는 것이 좋습니다.'

## RetrievalQA Chain

In [43]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [44]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [45]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [47]:
## RetrievalQA Chain을 어떻게 사용하는지 알아보자.
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
    
)

In [49]:
ai_message = qa_chain({"query":query})

In [50]:
ai_message

{'query': '연봉 5000만원의 직장인은 소득세가 얼마인가요?',
 'result': '연봉 5000만원의 직장인의 소득세는 소득세법에 따라 계산되며, 구체적인 세액은 개인의 공제 항목 및 세율에 따라 달라집니다. 일반적으로, 소득세는 누진세율로 적용되므로, 정확한 금액을 알기 위해서는 세무 전문가의 상담이 필요합니다. 따라서, 정확한 소득세 금액은 제공된 정보로는 알 수 없습니다.'}